In [ ]:
install.packages("BiocManager")
BiocManager::install("missMethyl")
library(missMethyl)

In [ ]:
BiocManager::install("clusterProfiler")
BiocManager::install("org.Mm.eg.db")

library(clusterProfiler)
library(org.Mm.eg.db)

More on GO analysis:

https://yulab-smu.top/biomedical-knowledge-mining-book/enrichment-overview.html

https://yulab-smu.top/biomedical-knowledge-mining-book/clusterprofiler-go.html

We can understand which biological processes, molecular functions and cell components can be influenced by differences in methylation level

In [ ]:
BiocManager::install(c("GenomicRanges", "rtracklayer", "biomaRt"))

In [12]:
dmr_data <- read.table("denovo_qval.0.05.out", header = FALSE)

library(GenomicRanges)

dmr_gr <- GRanges(seqnames = dmr_data$V1,
                  ranges = IRanges(start = dmr_data$V2, end = dmr_data$V3))
library(biomaRt)

ensembl <- useMart("ensembl")
mouse <- useDataset("mmusculus_gene_ensembl", mart = ensembl)

gene_annotation <- getBM(attributes = c("ensembl_gene_id", "external_gene_name", "chromosome_name", "start_position", "end_position"),
                         mart = mouse)

gene_gr <- GRanges(seqnames = gene_annotation$chromosome_name,
                   ranges = IRanges(start = gene_annotation$start_position, end = gene_annotation$end_position),
                   gene_id = gene_annotation$ensembl_gene_id,
                   gene_name = gene_annotation$external_gene_name)
annotated_genes <- findOverlaps(dmr_gr, gene_gr)

gene_ids <- gene_gr$gene_id[subjectHits(annotated_genes)]
gene_names <- gene_gr$gene_name[subjectHits(annotated_genes)]

annotated_dmr <- data.frame(
  chr = seqnames(dmr_gr)[queryHits(annotated_genes)],
  start = start(dmr_gr)[queryHits(annotated_genes)],
  end = end(dmr_gr)[queryHits(annotated_genes)],
  gene_id = gene_ids,
  gene_name = gene_names
)

head(annotated_dmr)


Warning message in .merge_two_Seqinfo_objects(x, y):
“The 2 combined objects have no sequence levels in common. (Use
  suppressWarnings() to suppress this warning.)”


chr,start,end,gene_id,gene_name
<fct>,<int>,<int>,<chr>,<chr>


TODO: fix this

In [20]:
DMRs <- read.table("denovo_qval.0.05.out", header = TRUE)
annotated_DMRs <- annotateRegions(DMRs,
                                  genome = "mm39",
                                  feature = "gene")
gene_info <- unique(annotated_DMRs$gene)
write.table(gene_info, "dmr_genes.txt", row.names = FALSE, col.names = TRUE)
gene_list <- read.table("dmr_genes.txt", header = TRUE)
ego <- enrichGO(gene = gene_list$gene,
                OrgDb = org.Mm.eg.db,
                keyType = "SYMBOL",
                ont = "BP",
                pAdjustMethod = "BH",
                qvalueCutoff = 0.05)
barplot(ego, showCategory=10)
enrichMap(go_results)


ERROR: Error in annotateRegions(DMRs, genome = "mm39", feature = "gene"): could not find function "annotateRegions"
